# Cleaning Pipeline: NYC Collisions Sample

**Author:** Yougi Jain  
**Date:** June 8, 2025  
**Project:** ds-fundamentals-ingest-clean  

## Setup

In [1]:
import pandas as pd
import re

In [2]:
raw_collisions = pd.read_csv('../data/raw/nyc_collisions_sample.csv')
pd.set_option("display.max_columns", None)
raw_collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,03/08/2022,23:00,NaN,NaN,NaN,NaN,NaN,LINDEN BOULEVARD,SOUTH CONDUIT AVENUE,NaN,1.0,0,0,0,0,0,1,0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,4508554,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,07/24/2021,18:51,MANHATTAN,10026.0,40.802480,-73.954850,"(40.80248, -73.95485)",NaN,NaN,234 WEST 114 STREET,0.0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,Unspecified,NaN,NaN,4444677,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN
2,01/22/2023,16:16,BROOKLYN,11221.0,40.689420,-73.921970,"(40.68942, -73.92197)",NaN,NaN,1366 BROADWAY,1.0,0,0,0,0,0,1,0,Steering Failure,Unspecified,Unspecified,NaN,NaN,4601423,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN
3,07/24/2021,21:20,NaN,NaN,40.751526,-73.859184,"(40.751526, -73.859184)",108 STREET,NaN,NaN,1.0,0,0,0,0,0,0,0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,4441397,E-Bike,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,03/31/2022,9:00,QUEENS,11377.0,40.756298,-73.898930,"(40.756298, -73.89893)",32 AVENUE,68 STREET,NaN,1.0,0,1,0,0,0,0,0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,4514997,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


### So let's develop a plan based on what we see above ^:

1. normalize column names converting stuff to snake_case and lower case
2. drop location, it's unneeded (latitude,longitude)
3. combine date and time
4. cast zipcode to integer
5. number of persons injured -> int
6. drop vehicle 4 and 5 related columns, almost entirely empty

## Cleaning

In [3]:
clean_columns = []
for col in raw_collisions.columns:
    c = col.strip()
    c = re.sub(r"[^\w\s]", "", c)
    c = re.sub(r"\s+","_",c).lower()
    clean_columns.append(c)
    
raw_collisions.columns = clean_columns
raw_collisions.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street_name', 'cross_street_name',
       'off_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'collision_id', 'vehicle_type_code_1', 'vehicle_type_code_2',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5'],
      dtype='object')

In [4]:
raw_collisions=raw_collisions.drop(columns=["location"])
raw_collisions.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'on_street_name', 'cross_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'collision_id', 'vehicle_type_code_1',
       'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4',
       'vehicle_type_code_5'],
      dtype='object')

In [5]:
raw_collisions["crash_datetime"] = pd.to_datetime(raw_collisions["crash_date"].astype(str)+" "+raw_collisions["crash_time"].astype(str))
raw_collisions = raw_collisions.drop(columns=["crash_date", "crash_time"])
raw_collisions[["crash_datetime"]].head()

,crash_datetime
0,2022-03-08 23:00:00
1,2021-07-24 18:51:00
2,2023-01-22 16:16:00
3,2021-07-24 21:20:00
4,2022-03-31 09:00:00


In [6]:
#let's keep the NaN columns for now, we don't know what we're gonna look at specifically yet
zip_numeric = pd.to_numeric(raw_collisions["zip_code"], errors="coerce")
mask = zip_numeric.notna()
(~mask).sum()
raw_collisions.loc[mask, "zip_code"] = (
    zip_numeric[mask]
    .astype(int)
    .astype(str)
    .str.zfill(5)
)
raw_collisions["zip_code"]

0          NaN
1        10026
2        11221
3          NaN
4        11377
         ...  
27159      NaN
27160    10461
27161    11205
27162      NaN
27163    11207
Name: zip_code, Length: 27164, dtype: object

In [8]:
#Check what we have so far
raw_collisions.head()

,borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,crash_datetime
0,NaN,NaN,NaN,NaN,LINDEN BOULEVARD,SOUTH CONDUIT AVENUE,NaN,1.0,0,0,0,0,0,1,0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,4508554,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2022-03-08 23:00:00
1,MANHATTAN,10026,40.802480,-73.954850,NaN,NaN,234 WEST 114 STREET,0.0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,Unspecified,NaN,NaN,4444677,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN,2021-07-24 18:51:00
2,BROOKLYN,11221,40.689420,-73.921970,NaN,NaN,1366 BROADWAY,1.0,0,0,0,0,0,1,0,Steering Failure,Unspecified,Unspecified,NaN,NaN,4601423,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,2023-01-22 16:16:00
3,NaN,NaN,40.751526,-73.859184,108 STREET,NaN,NaN,1.0,0,0,0,0,0,0,0,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,4441397,E-Bike,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2021-07-24 21:20:00
4,QUEENS,11377,40.756298,-73.898930,32 AVENUE,68 STREET,NaN,1.0,0,1,0,0,0,0,0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,4514997,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2022-03-31 09:00:00


In [ ]:
zip_numeric = pd.to_numeric(raw_collisions["number_of_persons_injured"], errors="coerce")
mask = zip_numeric.notna()
(~mask).sum()
raw_collisions.loc[mask, "number_of_persons_injured"] = (
    zip_numeric[mask]
    .astype(int)
)
raw_collisions["number_of_persons_injured"]